### Generating Porous Media Images with PoreSpy
PoreSpy is a toolkit for analyzing and generating images of porous materials. Here, we'll create binary images representing porous media.

In [ ]:
import numpy as np
import porespy as ps
import matplotlib.pyplot as plt

def generate_porous_image(image_size=128, porosity=0.4, blobiness=1, seed=0):
    """
    Generates a binary image of porous media using the blobs model in PoreSpy.

    Parameters:
    -----------
    image_size : int
        The dimensions of the image (e.g., 128 will produce a 128x128 image).
    porosity : float
        Desired porosity level of the generated image.
    blobiness : float
        Controls the morphology of the blobs (higher values create larger, more connected blobs).
    seed : int
        Seed for reproducibility.

    Returns:
    --------
    image : np.ndarray
        A binary image representing the porous media (1 = solid, 0 = pore space).
    """
    # Set random seed for reproducibility
    np.random.seed(seed)
    
    # Define the shape of the image
    shape = [image_size, image_size, 1]  # 2D image with single channel
    
    # Generate the porous media image
    image = ps.generators.blobs(shape=shape, porosity=porosity, blobiness=blobiness)
    
    """ 
    # Display the generated image
    plt.imshow(image.squeeze(), cmap='gray')
    plt.title(f"Generated Porous Media\nPorosity={porosity}, Blobiness={blobiness}")
    plt.axis('off')
    plt.show()
    
    """
    return image

# Example usage
generated_image = generate_porous_image(image_size=128, porosity=0.4, blobiness=1)


### Function to compute Diffusivity

In [ ]:
import openpnm as op
import porespy as ps
import numpy as np

def compute_diffusivity_from_image(image, voxel_size=1, C_in=10, C_out=5):
    """
    Computes effective diffusivity from a binary image using OpenPNM and PoreSpy.

    Parameters:
    -----------
    image : np.ndarray
        Binary image of porous media where 1 represents the solid phase and 0 represents the pore space.
    voxel_size : float, optional
        The size of each voxel in the image (default is 1).
    C_in : float, optional
        Concentration at the inlet (default is 10).
    C_out : float, optional
        Concentration at the outlet (default is 5).

    Returns:
    --------
    D_eff : float
        Computed effective diffusivity.
    """
    # Create a pore network model from the image
    snow = ps.networks.snow2(image, boundary_width=3, parallelization=None, voxel_size=voxel_size)
    net = snow.network

    # Convert the PoreSpy network to OpenPNM network
    pn = op.io.network_from_porespy(net)
    mods = op.models.collections.geometry.spheres_and_cylinders
    pn.add_model_collection(mods)

    # Map the pore and throat diameters
    pn['pore.diameter'] = pn['pore.equivalent_diameter']
    pn['throat.diameter'] = pn['throat.inscribed_diameter']
    pn['throat.spacing'] = pn['throat.total_length']

    # Check network health and remove disconnected pores
    h = op.utils.check_network_health(pn)
    op.topotools.trim(network=pn, pores=h['disconnected_pores'])
    pn.regenerate_models()

    # Define the phase (air) and add physical properties
    air = op.phase.Air(network=pn)
    phys = op.models.collections.physics.basic
    air.add_model_collection(phys)
    air.regenerate_models()

    # Define boundary conditions
    inlet = pn.pores('zmin')
    outlet = pn.pores('zmax')

    # Run a Fickian diffusion simulation
    fd = op.algorithms.FickianDiffusion(network=pn, phase=air)
    fd.set_value_BC(pores=inlet, values=C_in)  # Set boundary conditions
    fd.set_value_BC(pores=outlet, values=C_out)
    fd.run()

    # Calculate effective diffusivity
    rate_inlet = fd.rate(pores=inlet)[0]
    shape = image.shape
    A = (shape[1] * shape[2]) * (voxel_size ** 2)  # Cross-sectional area
    L = shape[0] * voxel_size  # Length of the sample
    D_eff = rate_inlet * L / (A * (C_in - C_out))

    print(f"Molar flow rate: {rate_inlet:.5e} mol/s")
    print(f"Effective Diffusivity: {D_eff:.6E} m^2/s")
    
    return D_eff


# Load or generate a binary image using PoreSpy or any other method
image = ps.generators.blobs(shape=[128, 128, 1], porosity=0.8)

# Compute effective diffusivity
D_eff = compute_diffusivity_from_image(image, voxel_size=1)
print(f"Computed Effective Diffusivity: {D_eff:.6E} m^2/s")



## Preparing Data for CNN Training
With images generated and effective diffusivity values calculated, you’ll need to prepare the data in a format suitable for training a CNN.

In [ ]:
# Example of creating datasets
iteration= 0
image_dict = {}
images = []  # Store generated images
diffusivity_values = []  # Store calculated diffusivity values
image_size = 128
porosity_range = np.linspace(0.3, 0.8, 1000)  # Generate porosities from 0.3 to 0.8
blobiness_range = range(1, 4)  # Blobiness from 1 to 3

for porosity in porosity_range:
    for blobiness in blobiness_range:
        image = generate_porous_image(image_size=image_size, porosity=porosity, blobiness=blobiness)
        image_dict[(porosity, blobiness)] = image  # Store image with (porosity, blobiness) as key
        D_eff = compute_diffusivity_from_image(image, voxel_size=1)
        images.append(image)
        diffusivity_values.append(D_eff)
        iteration = iteration+1
        print("**************************** {iteration}", iteration)
# Convert to NumPy arrays
images = np.array(images)
diffusivity_values = np.array(diffusivity_values)


### CNN Training and Validation
Use the CNN code from the previous response, substituting the generated images and diffusivity values as inputs for training and validation.

In [ ]:
# Normalize images and reshape
images = images.reshape(-1, image_size, image_size, 1) / 255.0  # Normalize to [0, 1]

# Split data into training, validation, and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, diffusivity_values, test_size=0.3, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Define and train the CNN model (use previous CNN code)
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Conv2D(32, (5, 5), activation='relu', input_shape=(image_size, image_size, 1)),
    MaxPooling2D((4, 4)),
    Conv2D(64, (5, 5), activation='relu'),
    MaxPooling2D((4, 4)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='linear')  # Output layer for regression
])

model.compile(optimizer=Adam(learning_rate=1e-4), loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), verbose=1)

# Evaluate the model
y_pred = model.predict(X_test)
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error on Test Set: {mse}")


### Prediction

In [ ]:
import numpy as np

def evaluate_model_on_new_image(model, new_image, image_size=128):
    """
    Evaluates the trained CNN model on a new image to predict diffusivity.

    Parameters:
    -----------
    model : tf.keras.Model
        The trained CNN model.
    new_image : np.ndarray
        The new binary image to evaluate, expected to be of size (image_size, image_size).
    image_size : int
        The size of the image (e.g., 128 for a 128x128 image).

    Returns:
    --------
    predicted_diffusivity : float
        Predicted diffusivity for the new image.
    """
    # Ensure the new image has the right shape and normalize it
    new_image = new_image.reshape(1, image_size, image_size, 1) / 255.0  # Add batch dimension and normalize
    
    # Make prediction
    predicted_diffusivity = model.predict(new_image)[0][0]  # Get the scalar prediction
    
    return predicted_diffusivity

# Example usage
# Assuming new_image is a binary image of size 128x128 (1s and 0s) and `model` is your trained CNN model
new_image = generate_porous_image(image_size=image_size, porosity=0.4, blobiness=3)
predicted_diffusivity = evaluate_model_on_new_image(model, new_image)
print(f"Predicted Diffusivity with AI model : {predicted_diffusivity}")
D_eff = compute_diffusivity_from_image(new_image, voxel_size=1)
print(f"Predicted Diffusivity with pore scale model : {D_eff}")
